In [1]:
require 'erb'
require 'rbplotly'

class String
  def to_df
    headers = lines.first.split(',').map(&:strip)
    rows = lines.drop(1).map { |line| line.split(',').map(&:strip).map(&:to_f) }
                          .map { |row| headers.zip(row).to_h }.to_a
    DF.new(headers, rows)
  end
end

class DF < Array
  attr_accessor :headers

  def initialize(headers, rows)
    @headers = headers
    super(rows)
  end  

  def to_html
    erb = ERB.new <<TEMPLATE
<table>
  <tr>
  <% self.headers.each do |header| %>
    <th><%= header %></th>
  <% end %>
  </tr>

  <% self.each do |row| %>
    <tr>
    <% self.headers.each do |header| %>
      <td><%= row[header] %></td>
    <% end %>
    </tr>
  <% end %>
</table>
TEMPLATE
    erb.result(binding)
  end

  def map
    resulting_arr = super
    self.class.new(headers, resulting_arr)
  end

  def to_plotly(targets: [{ x: 'x', y: 'y', name: nil }], type: :scatter, mode: :markers, layout: {})
    data = targets.map do |target|
      x_axis_name = target[:x]
      y_axis_name = target[:y]
      xs = self.map { |row| row[x_axis_name] }
      ys = self.map { |row| row[y_axis_name] }
      trace = { x: xs, y: ys, type: type, mode: mode, name: target[:name] }

      trace
    end

    Plotly::Plot.new(data: data, layout: layout)
  end
end

class Array
  def to_df
    headers = self.map { |row| row.keys }.flatten.uniq
    DF.new(headers, self)
  end
end

:to_df

In [2]:
jikken_b_1 = <<DATA.to_df
x, vh
0   , 48.181
0.01, 48.064
0.02, 47.871
0.03, 47.605
0.04, 47.401
0.05, 47.242
0.06, 47.102
0.07, 46.702
0.08, 46.612
0.09, 46.172
0.100, 45.651
0.105, 45.153
0.110, 44.489
0.115, 43.653
0.120, 42.527
0.125, 41.123
0.130, 39.102
0.135, 36.342
0.140, 32.784
0.145, 28.525
0.150, 23.331
0.155, 18.245
0.160, 14.124
0.165, 10.892
0.170, 8.687
0.175, 6.891
0.180, 5.655
0.185, 4.682
0.190, 3.973
0.195, 3.374
0.200, 2.947
0.21, 2.091
0.22, 1.642
0.23, 1.341
0.24, 1.142
0.25, 0.973
0.26, 0.851
0.27, 0.751
0.28, 0.682
0.29, 0.637
0.30, 0.594
0.31, 0.546
0.32, 0.517
0.33, 0.493
0.34, 0.474
0.35, 0.465
0.36, 0.447
0.37, 0.422
0.38, 0.413
0.39, 0.413
0.40, 0.404
DATA
nil

In [3]:
jikken_b_2 = <<DATA.to_df
y, vh
0,    -0.113
0.01, -0.137
0.02, -0.167
0.04, -0.223
0.06, -0.307
0.08, -0.427
0.10, -0.604
0.11, -0.730
0.12, -0.923
0.13, -1.159
0.14, -1.453
0.15, -1.901
0.16, -2.465
0.17, -3.072
0.18, -3.454
0.19, -3.092
0.195, -2.542
0.200, -1.950
0.205, -1.003
0.210, -0.125
0.215, 0.821
0.220, 1.694
0.225, 2.443
0.230, 3.030
0.235, 3.389
0.24, 3.464
0.25, 3.192
0.26, 2.619
0.27, 2.063
0.28, 1.637
0.29, 1.301
0.30, 1.062
0.31, 0.885
0.32, 0.754
0.34, 0.572
0.36, 0.446
0.38, 0.373
0.40, 0.310
DATA
nil

In [4]:
"""
def convert_vh_to_h(vh)
  i_mag = 1.00
  vh_at_imag_1 = 48.103 * 10 ** (-3)
  k = i_mag / vh_at_imag_1
  
  k * vh
end
"""

def convert_vh_to_h(vh)
  imag_to_vhmv = 48.103
  vhmv_to_imag = 1.0 / imag_to_vhmv
  imag = vh * vhmv_to_imag
  imag * 20000.0
end

:convert_vh_to_h

# 実験データ

## 実験(b)

In [5]:
jikken_b_1.map{ |row| row.merge('h' => convert_vh_to_h(row['vh'])) }.to_a.to_df.to_plotly(
  targets: [{ x: 'x', y: 'h' }],
  layout: {
    title: 'x-H',
    height: 400,
    xaxis: { title: 'x[m]' },
    yaxis: { title: 'H[A/m]' }
    }
).show

#<Plotly::Offline::HTML:0x007ffc6c34ace0 @id="4da133f0-1afc-482b-b127-d6b84f3a2e17", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[20032.430409745753, 19983.784795127125, 19903.54031973058, 19792.944307007878, 19708.126312288216, 19642.018169344945, 19583.809741596157, 19417.49994802819, 19380.080244475397, 19197.139471550632, 18980.520965428354, 18773.46527243623, 18497.391015113404, 18149.803546556348, 17681.641477662517, 17097.894102238944, 16257.613870236783, 15110.0762946178, 13630.750680830717, 11859.967153815769, 9700.434484335696, 7585.805459118975, 5872.398810884977, 4528.615678855788, 3611.832941812361, 2865.1019686921813, 2351.204706567158, 1946.6561337130743, 1651.8720246138494, 1402.8231087458162, 1225.2874041120097, 869.3844458765566, 682.7017025965117, 557.5535829366152, 474.81446063655073, 404.54857285408394, 353.82408581585344, 312.24663742386133, 283.5581980333867, 264.8483462569902, 246.97004344843356, 227.01286822027734, 214.95540818659958, 204.97682057252146, 197.07710537804294, 193.33513502276367, 185.85119431220508, 175.456832214207, 171.71486185892772, 171.71486185892772, 167.9728915036484], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"x-H", :height=>400, :xaxis=>{:title=>"x[m]"}, :yaxis=>{:title=>"H[A/m]"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

In [6]:
b_2_with_offset = jikken_b_2.map { |row| row.merge('y' => row['y'] - 0.21) }
b_2_with_offset.map{ |row| row.merge('h' => convert_vh_to_h(row['vh'])) }.to_a.to_df.to_plotly(
  targets: [{ x: 'y', y: 'h' }],
  layout: {
    title: 'y-H(with offset y=0.21)',
    height: 400,
    xaxis: { title: 'y[m]'  , dtick: 0.1 },
    yaxis: { title: 'H[A/m]' } }
).show

#<Plotly::Offline::HTML:0x007ffc6c421538 @id="9fc59fc2-059f-490e-b43b-82f196dfef29", @data=[{:x=>[-0.21, -0.19999999999999998, -0.19, -0.16999999999999998, -0.15, -0.13, -0.10999999999999999, -0.09999999999999999, -0.09, -0.07999999999999999, -0.06999999999999998, -0.06, -0.04999999999999999, -0.03999999999999998, -0.03, -0.01999999999999999, -0.014999999999999986, -0.009999999999999981, -0.0050000000000000044, 0.0, 0.0050000000000000044, 0.010000000000000009, 0.015000000000000013, 0.020000000000000018, 0.024999999999999994, 0.03, 0.04000000000000001, 0.05000000000000002, 0.060000000000000026, 0.07000000000000003, 0.07999999999999999, 0.09, 0.1, 0.11000000000000001, 0.13000000000000003, 0.15, 0.17, 0.19000000000000003], :y=>[-46.98251668295117, -56.961104297029294, -69.43433881462695, -92.71770991414256, -127.642766563416, -177.53570463380663, -251.1277882876328, -303.51537326154295, -383.7598486580879, -481.88262686318944, -604.1203251356465, -790.3872939317714, -1024.8841028626073, -1277.2592146019997, -1436.08506745941, -1285.5747042803982, -1056.8987381244413, -810.7602436438475, -417.0218073716816, -51.97181048999023, 341.35085129825586, 704.3219757603476, 1015.737064216369, 1259.7966862773633, 1409.059726004615, 1440.2428122986094, 1327.1521526723907, 1088.9133733862755, 857.7427603267988, 680.6228301769121, 540.9226035798183, 441.552501922957, 367.9604182691308, 313.49396087562104, 237.82300480219527, 185.43541982828512, 155.08388250213085, 128.8900900151758], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"y-H(with offset y=0.21)", :height=>400, :xaxis=>{:title=>"y[m]", :dtick=>0.1}, :yaxis=>{:title=>"H[A/m]"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

# 検討

## 磁場の変化を見る

In [7]:
b_1 = jikken_b_1
b_1_with_differential = b_1.to_a.zip(b_1.to_a.drop(1))[0...-1].map do |current_row, next_row|
  differential = (convert_vh_to_h(next_row['vh']) - convert_vh_to_h(current_row['vh']))/(next_row['x'] - current_row['x'])
  current_row.merge('differential' => differential)
end.to_df

b_1_with_differential.to_plotly(
  targets: [{x: 'x', y: 'differential'}],
  layout: {
    title: 'x-dH/dx',
    height: 400,
    xaxis: { title: 'x(m)' },
    yaxis: { title: 'dH/dx' }
  }
).show

#<Plotly::Offline::HTML:0x007ffc6abafc48 @id="75691c8d-7adb-4dbb-9d3a-36ead639dfff", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39], :y=>[-4864.5614618628315, -8024.447539654648, -11059.601272270085, -8481.799471966222, -6610.814294327064, -5820.842774878841, -16630.97935679651, -3741.970355279411, -18294.077292476515, -21661.850612227845, -41411.13859842452, -55214.851464565385, -69517.49371141117, -93632.41377876638, -116749.4750847144, -168056.04640043224, -229507.51512379627, -295865.12275741633, -354156.70540299127, -431906.53389601427, -422925.8050433438, -342681.32964679925, -268756.62640583754, -183356.54740868532, -149346.19462403658, -102779.45242500455, -80909.71457081671, -58956.82181984493, -49809.78317360659, -35507.14092676127, -35590.29582354538, -18668.27432800447, -12514.811965989646, -8273.912230006457, -7026.5887782466725, -5072.448703823046, -4157.744839199207, -2868.84393904746, -1870.9851776396563, -1787.8302808556653, -1995.71752281562, -1205.7460033677746, -997.8587614078114, -789.971519447851, -374.1970355279296, -748.3940710558578, -1039.4362097998073, -374.1970355279275, 0.0, -374.1970355279303], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"x-dH/dx", :height=>400, :xaxis=>{:title=>"x(m)"}, :yaxis=>{:title=>"dH/dx"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

$$
0.07 \lt x \lt 0.15 (m)
$$
において$H$は変化している

In [8]:
b_2_with_differential = b_2_with_offset.to_a.zip(b_2_with_offset.to_a.drop(1))[0...-1].map do |current_row, next_row|
  differential = (convert_vh_to_h(next_row['vh']) - convert_vh_to_h(current_row['vh']))/(next_row['y'] - current_row['y'])
  current_row.merge('differential' => differential)
end.to_df

b_2_with_differential.to_plotly(
  targets: [{x: 'y', y: 'differential'}],
  layout: {
    title: 'y-dH/dy(with offset y=0.21)',
    height: 400,
    xaxis: { title: 'y(m)' },
    yaxis: { title: 'dH/dy' }
  }
).show

#<Plotly::Offline::HTML:0x007ffc6aade468 @id="54a75f47-e627-4a8f-bfa3-ae4067a5b44c", @data=[{:x=>[-0.21, -0.19999999999999998, -0.19, -0.16999999999999998, -0.15, -0.13, -0.10999999999999999, -0.09999999999999999, -0.09, -0.07999999999999999, -0.06999999999999998, -0.06, -0.04999999999999999, -0.03999999999999998, -0.03, -0.01999999999999999, -0.014999999999999986, -0.009999999999999981, -0.0050000000000000044, 0.0, 0.0050000000000000044, 0.010000000000000009, 0.015000000000000013, 0.020000000000000018, 0.024999999999999994, 0.03, 0.04000000000000001, 0.05000000000000002, 0.060000000000000026, 0.07000000000000003, 0.07999999999999999, 0.09, 0.1, 0.11000000000000001, 0.13000000000000003, 0.15, 0.17], :y=>[-997.8587614078114, -1247.3234517597684, -1164.1685549757792, -1746.252832463673, -2494.6469035195328, -3679.604182691305, -5238.758497391018, -8024.4475396544985, -9812.277820510146, -12223.769827245691, -18626.69687961253, -23449.680893083565, -25237.51117393922, -15882.585285741068, 15051.036317901171, 45735.193231191355, 49227.69889611871, 78747.68725443355, 73009.99937633821, 78664.53235764915, 72594.2248924183, 62283.01769120423, 48811.9244121988, 29852.607945450476, 6236.617258798873, -11309.065962621857, -23823.877928611502, -23117.06130594764, -17711.993014988657, -13970.022659709442, -9937.010165686124, -7359.208365382613, -5446.645739350971, -3783.5478036712852, -2619.3792486955126, -1517.5768663077122, -1309.689624347752], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"y-dH/dy(with offset y=0.21)", :height=>400, :xaxis=>{:title=>"y(m)"}, :yaxis=>{:title=>"dH/dy"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

$$
-0.15 \lt y \lt 0.15 (m)
$$
において$H$は変化している

In [36]:
dh_dx_at_x_0 = p b_1_with_differential.find { |row| row['x'] == 0.145 }['differential']
dh_dy_at_y_0 = p b_2_with_differential.find { |row| row['y'] == 0.0 }['differential']
puts "div H = #{"%.2e" % (dh_dx_at_x_0 + dh_dy_at_y_0 * 2)}"

-431906.53389601427
78664.53235764915
div H = -2.75e+05


In [34]:
b_1_with_differential

x,vh,differential
0.0,48.181,-4864.5614618628315
0.01,48.064,-8024.447539654648
0.02,47.871,-11059.601272270085
0.03,47.605,-8481.799471966222
0.04,47.401,-6610.814294327064
0.05,47.242,-5820.842774878841
0.06,47.102,-16630.97935679651
0.07,46.702,-3741.970355279411
0.08,46.612,-18294.077292476515
0.09,46.172,-21661.850612227845


In [35]:
b_2_with_differential

y,vh,differential
-0.21,-0.113,-997.8587614078114
-0.19999999999999998,-0.137,-1247.3234517597684
-0.19,-0.167,-1164.1685549757792
-0.16999999999999998,-0.223,-1746.252832463673
-0.15,-0.307,-2494.6469035195328
-0.13,-0.427,-3679.604182691305
-0.10999999999999999,-0.604,-5238.758497391018
-0.09999999999999999,-0.73,-8024.4475396544985
-0.09,-0.923,-9812.277820510146
-0.07999999999999999,-1.159,-12223.769827245691


In [10]:
#require 'math'
a = 0.3/(Math::PI*6000*2)

7.957747154594767e-06

In [11]:
def h_by_x(x)
  n = 20000.0
  i = 1.0
  l = 0.15
  a = 0.025 #適当
  n*i/2.0*((x+l)/Math.sqrt(a**2+(x+l)**2) - (x-l)/(Math.sqrt(a**2+(x-l)**2)))
end

:h_by_x

In [12]:
hoge = jikken_b_1.map do |row|
  row.merge(
    'h_by_calc' => h_by_x(row['x']),
    'h_by_jikken' => convert_vh_to_h(row['vh']),
  )
end.to_df
hoge.to_plotly(
  targets: [{x: 'x', y: 'h_by_jikken', name: '実験'}, {x: 'x', y: 'h_by_calc', name: '計算'}],
  layout: {
    title: 'x-H',
    height: 400,
    xaxis: { title: 'x(m)' },
    yaxis: { title: 'H(A/m)' }
  }
).show

#<Plotly::Offline::HTML:0x007ffc6abbcc68 @id="89f8c60c-6a60-45e6-a9cb-8975e808c367", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[20032.430409745753, 19983.784795127125, 19903.54031973058, 19792.944307007878, 19708.126312288216, 19642.018169344945, 19583.809741596157, 19417.49994802819, 19380.080244475397, 19197.139471550632, 18980.520965428354, 18773.46527243623, 18497.391015113404, 18149.803546556348, 17681.641477662517, 17097.894102238944, 16257.613870236783, 15110.0762946178, 13630.750680830717, 11859.967153815769, 9700.434484335696, 7585.805459118975, 5872.398810884977, 4528.615678855788, 3611.832941812361, 2865.1019686921813, 2351.204706567158, 1946.6561337130743, 1651.8720246138494, 1402.8231087458162, 1225.2874041120097, 869.3844458765566, 682.7017025965117, 557.5535829366152, 474.81446063655073, 404.54857285408394, 353.82408581585344, 312.24663742386133, 283.5581980333867, 264.8483462569902, 246.97004344843356, 227.01286822027734, 214.95540818659958, 204.97682057252146, 197.07710537804294, 193.33513502276367, 185.85119431220508, 175.456832214207, 171.71486185892772, 171.71486185892772, 167.9728915036484], :type=>:scatter, :mode=>:markers, :name=>"実験"}, {:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[19727.878476642876, 19724.396092595834, 19713.655823455007, 19694.726929153567, 19665.871513340036, 19624.203768589992, 19565.061838248843, 19480.852328247485, 19358.863746642943, 19176.95331967701, 18894.643812099053, 18693.858012415178, 18434.073331053693, 18093.12973920816, 17639.61953653326, 17029.999876542515, 16207.32752868886, 15106.70479376748, 13676.954453290424, 11925.444400351154, 9965.457582448797, 8005.413860941284, 6253.7327865774105, 4823.696371488103, 3722.6709379481904, 2899.4770431503425, 2289.214084305742, 1834.9352140304386, 1493.0932128524832, 1232.2751986135427, 1030.3150883081912, 745.204978907571, 559.8318669155467, 433.6289446905639, 344.338312862108, 279.10078337556763, 230.13299597695158, 192.52731997212004, 163.07722898470044, 139.62169348764485, 120.66429373268738, 105.1438671505145, 92.29192229719408, 81.54143267797286, 72.4669894691099, 64.7446216417702, 58.124257645248974, 52.41049540162468, 47.448944145467294, 43.11637238821997, 39.31350000310307], :type=>:scatter, :mode=>:markers, :name=>"計算"}], @layout={:title=>"x-H", :height=>400, :xaxis=>{:title=>"x(m)"}, :yaxis=>{:title=>"H(A/m)"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

In [38]:
gosa_df = hoge.map do |row|
  zettai_gosa = (row['h_by_calc'] - row['h_by_jikken']).abs
  soutai_gosa = zettai_gosa / row['h_by_calc'] * 100.0
  row.merge('soutai_gosa' => soutai_gosa, 'zettai_gosa' => zettai_gosa)
end.to_a.to_df
gosa_df.to_plotly(
  targets: [{x: 'x', y: 'soutai_gosa'}],
  layout: {
    title: 'x-相対誤差',
    height: 400,
    xaxis: { title: 'x(m)' },
    yaxis: { title: '誤差 %' }
  }
  ).show

#<Plotly::Offline::HTML:0x007ffc6c19cd58 @id="93d3feed-2d84-42ca-bb24-5dd96036526f", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[1.5437642393400597, 1.315065370384953, 0.9632130030881961, 0.4986988558288795, 0.2148635971689137, 0.09077769964591348, 0.09582337895126077, 0.32520332864200313, 0.10959578056916523, 0.10526255937073054, 0.4545052777036705, 0.4258471417092393, 0.3434817846419642, 0.31323385265609016, 0.2382247589990542, 0.3986742583007775, 0.3102691758336447, 0.022317910466559277, 0.33782208325327057, 0.5490549814098108, 2.659417251244544, 5.241557889587624, 6.0977017839793675, 6.117314812277102, 2.9773782851975557, 1.1855611872964493, 2.7079434241824716, 6.088548458190001, 10.634219645137009, 13.840082988293553, 18.92356212350233, 16.663799958908726, 21.947631591236398, 28.57849776021834, 37.89184732013616, 44.94712911991704, 53.74765548669511, 62.18302808613234, 73.87970092378104, 89.68996840052417, 104.67533170629295, 115.90690391414475, 132.90814931171371, 151.37750691973397, 171.95431578132536, 198.6118848488766, 219.74807393930885, 234.7742296073927, 261.8939577085011, 298.2590657507234, 327.2651671573125], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"x-相対誤差", :height=>400, :xaxis=>{:title=>"x(m)"}, :yaxis=>{:title=>"誤差 %"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

In [39]:
gosa_df.to_plotly(
  targets: [{x: 'x', y: 'zettai_gosa'}],
  layout: {
    title: 'x-絶対誤差',
    height: 400,
    xaxis: { title: 'x[m]' },
    yaxis: { title: '誤差[A/m]' }
  }
  ).show

#<Plotly::Offline::HTML:0x007ffc6b0842a0 @id="8c40f52a-d985-4d18-9227-5f3e706f2d11", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[304.55193310287723, 259.3887025312906, 189.88449627557202, 98.21737785431105, 42.254798948179086, 17.814400754952658, 18.747903347313695, 63.35238021929399, 21.21649783245448, 20.186151873622293, 85.8771533293002, 79.60726002105366, 63.31768405971161, 56.67380734818653, 42.02194112925645, 67.8942256964292, 50.28634154792235, 3.3715008503204444, 46.20377245970667, 65.47724653538535, 265.0230981131008, 419.60840182230913, 381.33397569243334, 295.0806926323148, 110.8379961358296, 34.37507445816118, 61.99062226141632, 111.72091968263567, 158.77881176136611, 170.5479101322735, 194.97231580381845, 124.1794669689856, 122.86983568096502, 123.92463824605125, 130.47614777444272, 125.44778947851631, 123.69108983890186, 119.71931745174129, 120.48096904868626, 125.22665276934538, 126.30574971574617, 121.86900106976285, 122.66348588940551, 123.4353878945486, 124.61011590893304, 128.59051338099346, 127.7269366669561, 123.04633681258233, 124.26591771346042, 128.59848947070776, 128.65939150054533], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"x-絶対誤差", :height=>400, :xaxis=>{:title=>"x[m]"}, :yaxis=>{:title=>"誤差[A/m]"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">

In [53]:
jiba = (gosa_df.select { |row| row['x'] > 0.25 }.map { |row| row['zettai_gosa'] }.to_a).inject(:+)/15

124.59262367548776

In [56]:
hoge.map do |row|
  zettai_gosa = (row['h_by_calc'] - row['h_by_jikken']).abs - jiba
  soutai_gosa = zettai_gosa / row['h_by_calc'] * 100.0
  row.merge('soutai_gosa' => soutai_gosa, 'zettai_gosa' => zettai_gosa)
end.to_a.to_df.to_plotly(
  targets: [{x: 'x', y: 'soutai_gosa'}],
  layout: {
    title: 'x-相対誤差(地磁気の影響を引いた場合)',
    height: 400,
    xaxis: { title: 'x(m)' },
    yaxis: { title: '誤差 %' }
  }
  ).show

#<Plotly::Offline::HTML:0x007ffc6c1fc898 @id="6b3bab7f-e848-44c6-98af-cdf81fab4fe5", @data=[{:x=>[0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1, 0.105, 0.11, 0.115, 0.12, 0.125, 0.13, 0.135, 0.14, 0.145, 0.15, 0.155, 0.16, 0.165, 0.17, 0.175, 0.18, 0.185, 0.19, 0.195, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4], :y=>[0.9122081203027232, 0.6833977487726622, 0.33120124032195486, -0.1339203428209718, -0.41868383341900756, -0.5441149316409039, -0.5409884272445908, -0.31436121184181776, -0.5339989329743587, -0.5444372213949986, -0.20490182684151145, -0.24064248067230373, -0.3324004332376911, -0.3753845647838353, -0.46809786557595473, -0.33293246265466, -0.4584733787605305, -0.8024325918858167, -0.5731455163025871, -0.4957079598506342, 1.409172366404326, 3.685203329539457, 4.105409693359427, 3.534386408824313, -0.3694827657058306, -3.1115110716415026, -2.734650369454501, -0.7014800247132087, 2.289621826126132, 3.729303852620823, 6.8308901739851535, -0.05544202175189823, -0.30773310637256446, -0.15404539701868492, 1.7086463745644973, 0.30640035928448633, -0.39174470951405377, -2.531228411870156, -2.5212929189440927, 0.45410500189479913, 1.4197456325013391, -2.5903770514984195, -2.090256371375745, -1.419199715939955, 0.024138209098279056, 6.1748599406849385, 5.392435307471914, -2.950338192867948, -0.6885421117606524, 9.29082289008726, 10.344456292969518], :type=>:scatter, :mode=>:markers, :name=>nil}], @layout={:title=>"x-相対誤差(地磁気の影響を引いた場合)", :height=>400, :xaxis=>{:title=>"x(m)"}, :yaxis=>{:title=>"誤差 %"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7ffc6c8904b0 last_endpoint="tcp://127.0.0.1:62218">